### Importación de librerias

In [34]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import os, re
from gensim.parsing.porter import PorterStemmer
from nltk.corpus import stopwords

__paths to change__

In [35]:
# input files folder paths
documents_path = './datos/docs-raw-texts/'
queries_path = './datos/queries-raw-texts/'
relevance_judgements_path = 'relevance-judgments.tsv'
# output files folder path
output_path = './salida/'

### Read documents methods

In [36]:
def get_documents(path: str) -> list:
    """
    read raw text from naf documents located in the directory path
    """
    data = []
    for file in sorted(os.listdir(path)):
        if file.endswith(".naf"):
            tree = ET.parse(path + file)
            text = tree.find('raw').text
            header = tree.find('nafHeader')
            if header:
                desc = header.find('fileDesc')
                if desc:
                    title = desc.attrib.get('title')
                    text = title + ' ' + text if title else text
            data.append(text)
    return data

In [37]:
def remove_stopwords(document: str) -> list:
    """
    remove the english stop words from data
    """
    lower = document.lower()
    words = lower.split(' ')
    stop_words = stopwords.words('english')
    return [word for word in words if word not in stop_words]

In [38]:
def remove_nonlatin(document: str) -> str:
    """
    replace problematic characters
    """
    document = re.sub('\n', ' ', document)
    document = re.sub('[^a-zA-Z]|[0-9]', ' ', document)
    document = re.sub('\s+', ' ', document)
    return document

In [39]:
def preprocessing(document: str) -> list:
    """
    clean data by removing non-latin characters
    stem data sentences
    remove stop words from a document
    """
    porter = PorterStemmer()
    document = remove_nonlatin(document)
    document = porter.stem_sentence(document)
    document = remove_stopwords(document)
    return document

### indexes and doc-term matrix

In [40]:
def get_words_index(documents: pd.Series) -> pd.Index:
    """
    return a sorted index of every word in the texts
    """
    # get all words in all documents
    words = set()
    for document in documents:
        words.update(set(document))
    # sort the words
    sorted_words = sorted(list(words))
    # get index of sorted words
    words_frame = pd.DataFrame(sorted_words, columns=['data'])
    words_index = words_frame.set_index('data').index
    return words_index

In [41]:
def get_index_word(word: str, words_index: pd.Index) -> int:
    """
    return the provided word index
    """
    try: return words_index.get_loc(word)
    except: return -1

In [42]:
def get_doc_term(documents: pd.DataFrame, words_index: pd.Index) -> list:
    """
    return the document term matrix that indicate how many terms repeats in each document
    """
    doc_term = [[0]*len(words_index) for _ in range(len(documents))]
    for doc_index, document in documents.iterrows():
        for word in document.filtered:
            word_index = get_index_word(word, words_index)
            if word_index != -1:
                doc_term[doc_index][word_index] += 1
    return doc_term

## Representación vectorial ponderada tf.idf

In [43]:
def get_tf(doc_term: list) -> list:
    """
    return the ft score from each word in all the documents
    """
    return [[1+np.log10(word) if word > 0 else 0 for word in doc] for doc in doc_term]

In [44]:
def get_df(doc_term: list) -> list:
    """
    return the document frecuency in terms of in how many documents the term is repeted
    """
    df = [0]*len(doc_term[0])
    for doc in doc_term:
        for i, word in enumerate(doc):
            if word > 0:
                df[i] += 1
    return df

In [45]:
def get_idf(doc_term: list) -> list:
    """
    return the idf score from each word in the entire collection
    """
    doc_num = len(doc_term) # number of documents
    df = get_df(doc_term)
    return [np.log10(doc_num/dfs) if dfs > 0 else 0.0 for dfs in df]

In [46]:
def get_tfidf(doc_term: list, column: str) -> list:
    """
    ponderate the tf-idf scores multiping them
    """
    tf = get_tf(doc_term)
    idf = get_idf(doc_term)
    index = pd.concat([pd.DataFrame.from_dict({column:[[tfs * idf[i] for i, tfs in enumerate(doc)]]}) for doc in tf], ignore_index=True)
    return index

## Procesamiento

In [47]:
# Step 1: obtain the documents and convet it to dataframe
data = get_documents(documents_path)
documents = pd.DataFrame(data, columns=['data'])
documents.head()

,data
0,William Beaumont and the Human Digestion.\n\nW...
1,Selma Lagerlöf and the wonderful Adventures of...
2,Ferdinand de Lesseps and the Suez Canal.\n\nFe...
3,Walt Disney’s ‘Steamboat Willie’ and the Rise ...
4,Eugene Wigner and the Structure of the Atomic ...


In [48]:
# Step 2: apply the preprocessing method
documents['filtered'] = documents.data.apply(preprocessing)
documents.head()

,data,filtered
0,William Beaumont and the Human Digestion.\n\nW...,"[william, beaumont, human, digest, william, be..."
1,Selma Lagerlöf and the wonderful Adventures of...,"[selma, lagerl, f, wonder, adventur, niel, hol..."
2,Ferdinand de Lesseps and the Suez Canal.\n\nFe...,"[ferdinand, de, lessep, suez, canal, ferdinand..."
3,Walt Disney’s ‘Steamboat Willie’ and the Rise ...,"[walt, disnei, steamboat, willi, rise, mickei,..."
4,Eugene Wigner and the Structure of the Atomic ...,"[eugen, wigner, structur, atom, nucleu, eugen,..."


In [49]:
# Step 3: get word-index, doc-term, and the tfidf index
words_index = get_words_index(documents.filtered)
doc_term = get_doc_term(documents, words_index)
doc_proc = get_tfidf(doc_term, 'doc_vector')
doc_proc.head()

,doc_vector
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 1.2187979981117376, 0.0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [50]:
# queries
# Step 1: obtain the documents and convet it to dataframe
data = get_documents(queries_path)
queries = pd.DataFrame(data, columns=['data'])
queries.head()

,data
0,Fabrication of music instruments
1,famous German poetry
2,Romanticism
3,University of Edinburgh research
4,bridge construction


In [51]:
# Step 2: apply the preprocessing method
queries['filtered'] = queries.data.apply(preprocessing)
queries.head()

,data,filtered
0,Fabrication of music instruments,"[fabric, music, instrument]"
1,famous German poetry,"[famou, german, poetri]"
2,Romanticism,[romantic]
3,University of Edinburgh research,"[univers, edinburgh, research]"
4,bridge construction,"[bridg, construct]"


In [52]:
# Step 3: get word-index, doc-term, and the tfidf index
doc_term_queries = get_doc_term(queries, words_index)
quer_proc = get_tfidf(doc_term_queries, 'query_vector')
quer_proc.head()

,query_vector
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## similitud del coseno

In [53]:
def similitud_coseno(vector1:list,vector2:list) -> float:
    """
    Calcula la similitud entre dos vectores de documentos de acuerdo al ángulo entre estos
    haciendo uso de la función coseno

    Se hace uso de las funciones de numpy para poder calcular el coseno entre dos vectores,
    que por definición es el producto punto entre el vector1 y el vector2, dividido entre el 
    producto de las normas de ambos vectores

    Parameters
    ----------
    vector1 : list
        vector del primer documento a comparar
    vector2 : list
        vector del segundo documento a comparar

    Returns
    -------
    float
        La similitud coseno entre vector1 y vector2
    """
    producto_punto = np.dot(vector1,vector2)
    norma_1 = np.linalg.norm(vector1)
    norma_2 = np.linalg.norm(vector2)
    return producto_punto/(norma_1*norma_2)



## Procesamiento

In [54]:
def similitud_coseno_docs(query_vector:list) -> str:
    """
    Calcula la similitud coseno para cada uno de los documentos con respecto
    a una query realizada. A partir de los resultados obtenidos, elimina los
    que sean igual a 0, se ordenan descendentement y se genera el string 
    resultante en el formato especificado para el output de RRDV

    Parameters
    ----------
    query_vector : list
        documento como vector de la query respectiva

    Returns
    -------
    str 
        string resultante en el formato especificado para el output de RRDV con
        los documentos recuperados clasficados - ordenados por el puntaje de 
        similitud del coseno.
    """

    similitud = doc_proc['doc_vector'].apply(lambda x: similitud_coseno(x,query_vector))
    similitud = similitud[similitud>0]
    similitud = similitud.sort_values(ascending=False)
    output = ''
    for index, value in similitud.items():
        if output == '':
            output += f'd{index}:{value}'
        else:
            output += f',d{index}:{value}'
    return output

    


In [55]:
"""
Se le aplica la funcion de similitud coseno a cada una de las queries 
sobre todos los documentos. El resultado es escrito en el archivo de
salida
"""
q = quer_proc.query_vector.apply(similitud_coseno_docs)
f = open(output_path+"RRDV-consultas_resultados.txt", "w")
for i in range(len(q)):
    f.write(f'q{i+1} {q[i]}\n')
f.close()

## Evaluación de resultados

Usando las funciones descritas en `metricas.ipynb`:

In [56]:
def precision_at_k(relevance_query: list, k: int) -> float:
    if k > 0 and k <= len(relevance_query):
        return sum(relevance_query[:k]) / k
    else:
        return None

def recall_at_k(relevance_query: list, number_relevant_docs:int, k: int) -> float:
    if k > 0 and k <= len(relevance_query) and number_relevant_docs > 0:
        return sum(relevance_query[:k]) / number_relevant_docs
    else:
        return 0.0

def average_precision(relevance_query:list) -> float :
    relevant_docs = np.sum(relevance_query)
    last_recall = 0
    precisions = np.array([])
    for i in range(1,len(relevance_query)+1):
        recall = recall_at_k(relevance_query,relevant_docs,i)
        if recall > last_recall:
            last_recall = recall 
            precisions= np.append(precisions,precision_at_k(relevance_query,i))
    if len(precisions) > 0:
        return np.average(precisions)
    else:
        return 0

def mean_average_precision(list_vectors: list)-> float:
    average_precisions = list(map(average_precision, list_vectors))
    return np.average(average_precisions)

def dcg_at_k(rel, k):
    import math
    result = 0
    for i in range(k):
        discount_factor = 1/math.log(max([i+1, 2]), 2)
        gain = + (rel[i]*discount_factor)
        result = result + gain 
    return result
    
def ndcg_at_k(rel, k):
    try:
        DCG = dcg_at_k(rel, k)
        IDCG = dcg_at_k(sorted(rel, reverse=True), k)
        result = DCG/IDCG
        return result
    except:
        return None

In [57]:
def get_relevance_judgements(path: str) -> list:
    """
    Transforma el documento de juicios de relevancia a una lista de diccionarios por consulta

    Parameters
    ----------
    path : str
        Ubicación del archivo (incluyendo nombre del archivo)

    Returns
    -------
    list
        Lista de diccionarios, un diccionario representa los documentos relevantes para una consulta
    """
    relevance_dicts = []
    for line in open(path, "r"):
        line_proc = line.split("\n")[0]
        docs_str = line_proc.split("\t")[1]
        docs_list = docs_str.split(",")
        current_relevance_dict = {}
        for doc in docs_list:
            doc_split = doc.split(":")
            doc_id = doc_split[0].split("d")[1]
            doc_relevance = doc_split[1]
            current_relevance_dict[doc_id] = doc_relevance
        relevance_dicts.append(current_relevance_dict)
    return relevance_dicts

In [58]:
def get_result_relevances(path: str, relevance_lists: list) -> list:
    """
    Retorna una lista donde los resultados de cada consulta se representan como una lista
    con la relevancia entera.

    Parameters
    ----------
    path : str
        Ubicación del archivo de resultados (incluyendo nombre del archivo)
    relevance_lists : list
        Lista de diccionarios obtenida con la función get_relevance_judgements()

    Returns
    -------
    list
        Lista de listas, cada lista representa los resultados para una consulta con relevancia entera
    """
    res = []
    for i, line in enumerate(open(path, "r")):
        line_proc = line.split("\n")[0]
        if len(line_proc.split("\t")) == 0:
            docs_str = line_proc.split("\t")[1]
        else:
            docs_str = line_proc.split(" ")[1]
        docs_list = docs_str.split(",")
        current_relevance_list = []
        for doc in docs_list:
            doc_split = doc.split(":")
            doc_id = doc_split[0].split("d")[1]
            if doc_id in relevance_lists[i]:
                current_relevance_list.append(int(relevance_lists[i][doc_id]))
            else:
                current_relevance_list.append(0)
        res.append(current_relevance_list)
    return res

In [59]:
def get_binary_result_relevances(path: str, relevance_lists: list) -> list:
    """
    Retorna una lista donde los resultados de cada consulta se representan como una lista
    con la relevancia binaria.

    Parameters
    ----------
    path : str
        Ubicación del archivo de resultados (incluyendo nombre del archivo)
    relevance_lists : list
        Lista de diccionarios obtenida con la función get_relevance_judgements()

    Returns
    -------
    list
        Lista de listas, cada lista representa los resultados para una consulta con relevancia binaria
    """
    res = []
    for i, line in enumerate(open(path, "r")):
        line_proc = line.split("\n")[0]
        if len(line_proc.split("\t")) == 0:
            docs_str = line_proc.split("\t")[1]
        else:
            docs_str = line_proc.split(" ")[1]
        docs_list = docs_str.split(",")
        current_relevance_list = []
        if docs_list[0] != '':
            for doc in docs_list:
                doc_split = doc.split(":")
                doc_id = doc_split[0].split("d")[1]
                if doc_id in relevance_lists[i]:
                    current_relevance_list.append(1)
                else:
                    current_relevance_list.append(0)
        res.append(current_relevance_list)
    return res

In [60]:
def precision_list(binary_result_relevances: list, relevance_lists: list) -> list:
    """
    Retorna una lista con el valor de P@M para cada consulta.

    Parameters
    ----------
    binary_result_relevances : list
        Lista de relevancias binarias para cada consulta
    relevance_lists : list
        Lista de diccionarios obtenida con la función get_relevance_judgements()

    Returns
    -------
    list
        Lista de float con los valores de P@M por consulta
    """
    p_list = []
    for i, query in enumerate(binary_result_relevances):
        p_list.append(precision_at_k(query,len(relevance_lists[i])))
    return p_list

In [61]:
def recall_list(binary_result_relevances: list, relevance_lists: list) -> list:
    """
    Retorna una lista con el valor de R@M para cada consulta.

    Parameters
    ----------
    binary_result_relevances : list
        Lista de relevancias binarias para cada consulta
    relevance_lists : list
        Lista de diccionarios obtenida con la función get_relevance_judgements()

    Returns
    -------
    list
        Lista de float con los valores de R@M por consulta
    """
    r_list = []
    for i, query in enumerate(binary_result_relevances):
        r_list.append(recall_at_k(query,len(relevance_lists[i]),len(relevance_lists[i])))
    return r_list

In [62]:
def ndcg_list(result_relevances: list, relevance_lists: list) -> list:
    """
    Retorna una lista con el valor de NDCG@M para cada consulta.

    Parameters
    ----------
    result_relevances : list
        Lista de relevancias enteras para cada consulta
    relevance_lists : list
        Lista de diccionarios obtenida con la función get_relevance_judgements()

    Returns
    -------
    list
        Lista de float con los valores de NDCG@M por consulta
    """
    ndcg_l = []
    for i, query in enumerate(result_relevances):
        ndcg_l.append(ndcg_at_k(query,len(relevance_lists[i])))
    return ndcg_l

In [63]:
"""
Se obtienen los juicios de relevancia desde el archivo
"""
relevance_dicts = get_relevance_judgements(relevance_judgements_path)
relevance_dicts

[{'186': '4', '254': '5', '016': '5'},
 {'136': '2',
  '139': '2',
  '143': '4',
  '283': '4',
  '228': '4',
  '164': '4',
  '318': '2',
  '291': '4',
  '293': '4',
  '147': '2',
  '149': '2'},
 {'152': '3', '291': '4', '283': '4', '147': '3', '318': '2', '105': '2'},
 {'275': '3',
  '010': '3',
  '286': '2',
  '019': '2',
  '049': '2',
  '330': '2',
  '270': '3'},
 {'069': '2', '233': '3', '257': '2', '297': '3', '026': '4', '329': '5'},
 {'004': '3', '077': '3', '266': '2', '179': '3'},
 {'205': '2',
  '005': '4',
  '110': '4',
  '108': '3',
  '117': '3',
  '081': '2',
  '292': '2',
  '251': '5',
  '028': '3',
  '271': '3',
  '121': '2',
  '180': '2'},
 {'205': '3', '199': '5', '198': '3', '223': '2', '217': '2', '177': '2'},
 {'068': '2',
  '100': '2',
  '065': '3',
  '076': '3',
  '231': '4',
  '199': '4',
  '052': '2',
  '215': '2'},
 {'239': '4', '277': '4', '258': '3', '250': '4'},
 {'239': '2', '277': '2', '258': '2', '049': '4', '056': '4'},
 {'002': '2',
  '005': '3',
  '142'

In [64]:
"""
Se obtiene la lista de relevancias binarias para las consultas
"""
binary_result_relevances = get_binary_result_relevances(output_path+"RRDV-consultas_resultados.txt", relevance_dicts)
binary_result_relevances

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [65]:
"""
Se obtiene la lista de relevancias enteras para las consultas
"""
result_relevances = get_result_relevances(output_path+"RRDV-consultas_resultados.txt", relevance_dicts)
result_relevances

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  4,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [66]:
"""
Se calculan las métricas para todas las consultas. El resultado es escrito en el archivo de
salida para métricas
"""
precision = precision_list(binary_result_relevances, relevance_dicts)
recall = recall_list(binary_result_relevances, relevance_dicts)
ndcg = ndcg_list(result_relevances, relevance_dicts)
f = open(output_path+"RRDV-metricas.txt", "w")
for i in range(len(precision)):
    f.write(f'q{i+1} P@M:{precision[i]}, R@M:{recall[i]}, NDCG@M:{ndcg[i]}\n')
f.write(f'\nMAP:{mean_average_precision(binary_result_relevances)}')
f.close()